In [1]:
import logging
from logging import getLogger

import pandas as pd
import pyarrow.dataset as ds
import torch
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.data.interaction import Interaction
from recbole.model.sequential_recommender import GRU4Rec
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger


2025-10-22 14:36:20.248464: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-22 14:36:20.248486: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-22 14:36:20.249353: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-22 14:36:20.254977: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-22 14:36:21.129504: W tensorflow/compiler/tf2

In [2]:
saved_model_path = "saved/GRU4Rec-Oct-22-2025_11-13-21.pth"
checkpoint = torch.load(saved_model_path, weights_only=False)

config = checkpoint["config"]
init_seed(config['seed'], config['reproducibility'])
init_logger(config)
dataset = create_dataset(config)

model = GRU4Rec(config, dataset).to(config['device'])

model.load_state_dict(checkpoint["state_dict"])

model.eval()

/home/hygo2025/Development/projects/kepler/.venv/lib/python3.9/site-packages/recbole/data/dataset/dataset.py:648: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  feat[field].fillna(value=0, inplace=True)
/home/hygo2025/Development/projects/kepler/.venv/lib/python3.9/site-packages/recbole/data/dataset/dataset.py:650: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because th

GRU4Rec(
  (item_embedding): Embedding(7403, 64, padding_idx=0)
  (emb_dropout): Dropout(p=0.3, inplace=False)
  (gru_layers): GRU(64, 128, bias=False, batch_first=True)
  (dense): Linear(in_features=128, out_features=64, bias=True)
  (loss_fct): CrossEntropyLoss()
)

In [4]:
base_path_events = "/home/hygo2025/Documents/data/processed_data/enriched_events_vix"
df_events = ds.dataset(base_path_events, format="parquet", partitioning="hive").to_table().to_pandas()

print(df_events.shape)
print(df_events.columns.tolist())
df_events.head()

(3921288, 20)
['listing_id', 'user_id', 'session_id', 'event_type', 'price', 'event_ts', 'event_id', 'platform', 'business_type', 'neighborhood', 'usable_areas', 'total_areas', 'bathrooms', 'bedrooms', 'suites', 'parking_spaces', 'amenities', 'usage_types', 'geopoint', 'dt']


,listing_id,user_id,session_id,event_type,price,event_ts,event_id,platform,business_type,neighborhood,usable_areas,total_areas,bathrooms,bedrooms,suites,parking_spaces,amenities,usage_types,geopoint,dt
0,28938,13991,17870,RankingRendered,480000.0,2024-02-16 14:06:37.476,ranking-rendered-54bx5i,DESKTOP,SALE,jardim camburi,46.0,46.0,1.0,1.0,0.0,1.0,"['ELEVATOR', 'FURNISHED', 'GARAGE']",['RESIDENTIAL'],"-40.267894,-20.2599804",2024-02-16
1,28938,13991,17870,RankingRendered,480000.0,2024-02-16 14:06:37.476,ranking-rendered-54bx5i,DESKTOP,RENTAL,jardim camburi,46.0,46.0,1.0,1.0,0.0,1.0,"['ELEVATOR', 'FURNISHED', 'GARAGE']",['RESIDENTIAL'],"-40.267894,-20.2599804",2024-02-16
2,28938,13991,17870,RankingRendered,480000.0,2024-02-16 14:06:37.476,ranking-rendered-54bx5i,DESKTOP,SALE,jardim camburi,46.0,46.0,1.0,1.0,0.0,1.0,"['ELEVATOR', 'FURNISHED', 'GARAGE']",['RESIDENTIAL'],"-40.267894,-20.2599804",2024-02-16
3,28938,13991,17870,RankingRendered,480000.0,2024-02-16 14:06:37.476,ranking-rendered-54bx5i,DESKTOP,RENTAL,jardim camburi,46.0,46.0,1.0,1.0,0.0,1.0,"['ELEVATOR', 'FURNISHED', 'GARAGE']",['RESIDENTIAL'],"-40.267894,-20.2599804",2024-02-16
4,28938,13991,17870,RankingRendered,480000.0,2024-02-16 14:06:40.770,ranking-rendered-pm1rwf,DESKTOP,SALE,jardim camburi,46.0,46.0,1.0,1.0,0.0,1.0,"['ELEVATOR', 'FURNISHED', 'GARAGE']",['RESIDENTIAL'],"-40.267894,-20.2599804",2024-02-16


In [10]:
df_events[df_events["session_id"] == 999]["listing_id"].unique()

array([51056,  6544, 14659, 38551, 18610, 18769, 51463, 25039, 22540,
       24835,  2594, 33352, 29675, 29556, 27779, 15530,  6318, 38449,
       43717, 48208, 46510, 43482, 10039, 36008, 53242, 20560, 18923,
       17818, 47531, 54058, 26566, 11117, 54358, 48528, 34513],
      dtype=int32)

In [3]:
base_path = "/home/hygo2025/Documents/data/processed_data/listings"

dataset = ds.dataset(base_path, format="parquet", partitioning="hive")
table = dataset.to_table()

df = table.to_pandas()

print(df.shape)
print(df.columns.tolist())
df.head()




(56332, 18)
['state', 'city', 'neighborhood', 'anonymized_listing_id', 'dt', 'created_at', 'updated_at', 'price', 'usable_areas', 'total_areas', 'bathrooms', 'bedrooms', 'suites', 'parking_spaces', 'amenities', 'usage_types', 'listing_id_numeric', 'geopoint']


,state,city,neighborhood,anonymized_listing_id,dt,created_at,updated_at,price,usable_areas,total_areas,bathrooms,bedrooms,suites,parking_spaces,amenities,usage_types,listing_id_numeric,geopoint
0,espírito santo,vitória,jardim camburi,FFB56014232347F6FC2B1FA258C1829CB6192FF094C389...,2024-07-01,2023-11-07 08:10:03,2023-11-07 08:10:06,4000.0,280.0,280.0,4.0,4.0,2.0,2.0,"['KITCHEN_CABINETS', 'BARBECUE_GRILL', 'ALARM_...",['RESIDENTIAL'],56332,"-40.267894,-20.2599804"
1,espírito santo,vitória,jardim camburi,FF6899146175A3F8624EC399B6A01D03D5529141B6BFA1...,2024-07-01,2022-11-09 19:09:46,2022-11-09 19:09:47,386360.0,47.0,47.0,1.0,1.0,0.0,NaN,"['GYM', 'BALCONY', 'SERVICE_AREA']",['RESIDENTIAL'],56331,"-40.267894,-20.2599804"
2,espírito santo,vitória,jardim camburi,FECA847FD06BF87B05AE788EE40EBBDE8404131397E1BC...,2024-07-01,2020-10-14 18:24:09,2021-01-11 15:08:24,540000.0,92.0,NaN,2.0,3.0,1.0,1.0,"['GYM', 'BARBECUE_GRILL', 'GOURMET_SPACE', 'PO...",['RESIDENTIAL'],56330,"-40.267894,-20.2599804"
3,espírito santo,vitória,jardim camburi,FE7CFD486CAC742EC229D21A5D675777F6EC7E11ABB624...,2024-07-01,2021-05-13 06:17:49,2021-05-13 06:17:51,1000.0,55.0,55.0,1.0,2.0,NaN,1.0,['SECURITY_24_HOURS'],['RESIDENTIAL'],56329,"-40.267894,-20.2599804"
4,espírito santo,vitória,jardim camburi,FC6C19FB5EF1C0BA57877F9E264BF909A58FC9CA49B056...,2024-07-01,2023-04-21 04:47:01,2023-04-21 04:47:02,1230000.0,240.0,300.0,2.0,3.0,1.0,2.0,[],['RESIDENTIAL'],56328,"-40.267894,-20.2599804"


In [14]:
from src.utils.similarity_calc import PropertySimilarityCalculator


In [19]:

numerical_cols = ['price', 'usable_areas', 'total_areas', 'bathrooms', 'bedrooms', 'suites', 'parking_spaces']
categorical_cols = ['state', 'city', 'neighborhood']

weights = {
    'price': 3.0,
    'usable_areas': 2.0,
    'total_areas': 2.0,
    # 'neighborhood': 5.0,
    # 'city': 4.0,
    # 'state': 2.0,
    'bathrooms': 1.0,
    'bedrooms': 2.0,
    'suites': 1.0,
    'amenities': 1.5
}


calculator = PropertySimilarityCalculator(
    dataframe=df,
    id_column='listing_id_numeric',
    numerical_features=numerical_cols,
    #categorical_features=categorical_cols,
    categorical_features=[],
    amenities_column='amenities'
)


viewed = df_events[df_events["session_id"] == 999]["listing_id"].unique()
recs = [54358,48208,6318,27709,54058,22540,10039,26566,29556,34513]

for listing_viewed in viewed:
    df_analise = calculator.analyze_recommendations(listing_viewed, recs, weights)
    print(f"--- Análise de Recomendações para o Imóvel Base ID {listing_viewed} ---")
    print(df_analise)
    print("--------------------------------------------------")
# imovel_base = 51056
# recomendacoes = [54358,48208,6318,27709,54058,22540,10039,26566,29556,34513]
#
# df_analise = calculator.analyze_recommendations(imovel_base, recomendacoes, weights)
#
# print(f"--- Análise de Recomendações para o Imóvel Base ID {imovel_base} ---")
# print(df_analise)

Calculador de Similaridade pronto!
--- Análise de Recomendações para o Imóvel Base ID 51056 ---
   recommended_id  similarity_score
0           10039          0.902652
1           26566          0.898000
2           34513          0.888654
3           48208          0.878364
4           29556          0.877572
5           22540          0.875552
6           54358          0.874970
7            6318          0.872106
8           27709          0.868654
9           54058          0.866652
--------------------------------------------------
--- Análise de Recomendações para o Imóvel Base ID 6544 ---
   recommended_id  similarity_score
0           54058          0.994653
1           54358          0.876655
2           27709          0.876654
3           34513          0.876654
4           22540          0.874662
5           29556          0.874662
6           48208          0.874656
7            6318          0.874652
8           10039          0.874651
9           26566          0.871999
-

In [23]:
import pandas as pd

numerical_cols = ['price', 'usable_areas', 'total_areas', 'bathrooms', 'bedrooms', 'suites', 'parking_spaces']
categorical_cols = ['state', 'city', 'neighborhood']

weights = {
    'price': 3.0,
    'usable_areas': 2.0,
    'total_areas': 2.0,
    'bathrooms': 1.0,
    'bedrooms': 2.0,
    'suites': 1.0,
    'amenities': 1.5
}

calculator = PropertySimilarityCalculator(
    dataframe=df,
    id_column='listing_id_numeric',
    numerical_features=numerical_cols,
    categorical_features=categorical_cols,
    amenities_column='amenities'
)

viewed = df_events[df_events["session_id"] == 999]["listing_id"].unique()
recs = [54358, 48208, 6318, 27709, 54058, 22540, 10039, 26566, 29556, 34513]

columns_to_view = numerical_cols + categorical_cols

for listing_viewed in viewed:
    df_analysis = calculator.analyze_recommendations(listing_viewed, recs, weights)
    if df_analysis.empty:
        print(f"No valid recommendations found to compare with {listing_viewed}.")
        continue

    print(df_analysis)

    top_3_similar_ids = df_analysis.head(5)['recommended_id'].tolist()

    if not top_3_similar_ids:
        print(f"\nNo similar items found among the recommendations for {listing_viewed}.")
        continue

    ids_to_compare = [listing_viewed] + top_3_similar_ids

    try:
        print(f"\n--- Feature Comparison: Base ({listing_viewed}) vs. Top 3 Similar ---")

        existing_columns = [col for col in columns_to_view if col in calculator.df.columns]

        df_comparison = calculator.df.loc[ids_to_compare, existing_columns].copy()

        df_comparison['Type'] = 'Recommended (Similar)'
        df_comparison.loc[listing_viewed, 'Type'] = '>>> VIEWED (BASE) <<<'

        final_columns = ['Type'] + existing_columns

        print(df_comparison[final_columns].to_markdown(numalign="left", stralign="left"))

    except KeyError as e:
        print(f"Error fetching data: ID {e} was not found in the DataFrame.")
    except Exception as e:
        print(f"An error occurred during feature comparison: {e}")

print("\n--- End of Analysis ---")

#estudo comparativo
#melhoria de um algoritmo

#olhar uma revista para tal a3 a4

Calculador de Similaridade pronto!
   recommended_id  similarity_score
0           10039          0.902652
1           26566          0.898000
2           34513          0.888654
3           48208          0.878364
4           29556          0.877572
5           22540          0.875552
6           54358          0.874970
7            6318          0.872106
8           27709          0.868654
9           54058          0.866652

--- Feature Comparison: Base (51056) vs. Top 3 Similar ---
| listing_id_numeric   | Type                  | price   | usable_areas   | total_areas   | bathrooms   | bedrooms   | suites   | parking_spaces   | state          | city    | neighborhood   |
|:---------------------|:----------------------|:--------|:---------------|:--------------|:------------|:-----------|:---------|:-----------------|:---------------|:--------|:---------------|
| 51056                | >>> VIEWED (BASE) <<< | 2000    | 60             | 60            | 1           | 2          | 0   